In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
from absbox.local.generic import Generic

test01 = Generic(
    "Pay Prin Seq"
    ,{"collect":["2022-05-01","2022-06-01"] 
      ,"pay":["2022-06-15","2022-07-15"] # next distribution date
      ,"payFreq":["DayOfMonth",20]
      ,"poolFreq":"MonthEnd"
      ,"stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]
      ,'issuanceStat':{"AccruedInterest":23.35}
     }
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":500
             ,"rate":0.07
             ,"originBalance":500
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("A2",{"balance":500
             ,"rate":0.07
             ,"originBalance":500
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.00
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         #,["accrueAndPayInt","acc01",["A1","A2"]]
         ,["accrueAndPayInt","acc01",["A1","A2"],{"limit":{"formula":("constant",2)}}]
         #,["payPrinBySeq","acc01",["A1","A2"]]
         ,["payPrinBySeq","acc01",["A1","A2"],{"limit":{"formula":("constant",100)}}]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedCash","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,"Amortizing"
    )

In [21]:
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)
#localAPI = API("https://absbox.org/api/dev",lang='english',check=False)

✅Connected, local lib:0.20.2, server:0.22.8

In [25]:
r = localAPI.run(test01
               ,poolAssump = ("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                       ,None
                                       ,None)
               ,runAssump = [("pricing"
                              ,{"date":"2022-07-20"
                                ,"curve":[["2021-01-01",0.025]
                                         ,["2024-08-01",0.025]]})]
               ,read=True)

In [26]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-05-01,1777.15,0.00,0.00,0,0.00,0,0.00,0.00,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-01,1562.58,211.69,0.00,0,2.88,0,2.88,0.08,None,None,211.69,0.0,0.0,2.88,0.0,2.88
2022-07-31,1454.52,106.77,10.00,0,1.29,0,1.29,0.08,None,None,318.46,0.0,0.0,4.17,0.0,4.17
2022-08-31,1345.89,107.39,9.68,0,1.24,0,1.24,0.08,None,None,425.85,0.0,0.0,5.41,0.0,5.41
2022-09-30,1236.74,108.01,8.96,0,1.14,0,1.14,0.08,None,None,533.86,0.0,0.0,6.55,0.0,6.55
2022-10-31,1127.07,108.65,8.23,0,1.02,0,1.02,0.08,None,None,642.51,0.0,0.0,7.57,0.0,7.57
2022-11-30,1016.83,109.28,7.50,0,0.96,0,0.96,0.08,None,None,751.79,0.0,0.0,8.53,0.0,8.53
2022-12-31,906.09,109.91,6.77,0,0.83,0,0.83,0.08,None,None,861.70,0.0,0.0,9.36,0.0,9.36
2023-01-31,794.77,110.55,6.03,0,0.77,0,0.77,0.08,None,None,972.25,0.0,0.0,10.13,0.0,10.13


In [73]:
r['bonds']['A2']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2022-07-15,500,1.00,0,0.07,1.00,"[<PayInt:A2>, <PayPrin:A2>]"
2022-07-20,500,0.00,0,0.07,0.00,"[<PayInt:A2>, <PayPrin:A2>]"
2022-08-20,500,1.06,0,0.07,1.06,"[<PayInt:A2>, <PayPrin:A2>]"
2022-09-20,500,1.13,0,0.07,1.13,"[<PayInt:A2>, <PayPrin:A2>]"
2022-10-20,500,1.21,0,0.07,1.21,"[<PayInt:A2>, <PayPrin:A2>]"
2022-11-20,500,1.31,0,0.07,1.31,"[<PayInt:A2>, <PayPrin:A2>]"
2022-12-20,400,1.42,100,0.07,101.42,"[<PayInt:A2>, <PayPrin:A2>]"
2023-01-20,300,1.50,100,0.07,101.50,"[<PayInt:A2>, <PayPrin:A2>]"
2023-02-20,200,1.55,100,0.07,101.55,"[<PayInt:A2>, <PayPrin:A2>]"


In [50]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2022-07-15,301.10,2.87,198.90,0.07,201.77,"[<PayInt:A1>, <PayPrin:A1>]"
2022-07-20,301.10,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2022-08-20,189.44,2.07,111.66,0.07,113.73,"[<PayInt:A1>, <PayPrin:A1>]"
2022-09-20,76.46,1.12,112.98,0.07,114.10,"[<PayInt:A1>, <PayPrin:A1>]"
2022-10-20,0.00,0.43,76.46,0.07,76.89,"[<PayInt:A1>, <PayPrin:A1>]"
2022-11-20,0.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2022-12-20,0.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2023-01-20,0.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2023-02-20,0.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"


In [51]:
r['bonds']['A2']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2022-07-15,500.00,2.87,0.00,0.07,2.87,"[<PayInt:A2>, <PayPrin:A2>]"
2022-07-20,500.00,0.00,0.00,0.07,0.00,"[<PayInt:A2>, <PayPrin:A2>]"
2022-08-20,500.00,3.44,0.00,0.07,3.44,"[<PayInt:A2>, <PayPrin:A2>]"
2022-09-20,500.00,2.97,0.00,0.07,2.97,"[<PayInt:A2>, <PayPrin:A2>]"
2022-10-20,462.79,2.87,37.21,0.07,40.08,"[<PayInt:A2>, <PayPrin:A2>]"
2022-11-20,348.66,2.75,114.13,0.07,116.88,"[<PayInt:A2>, <PayPrin:A2>]"
2022-12-20,233.88,2.00,114.78,0.07,116.78,"[<PayInt:A2>, <PayPrin:A2>]"
2023-01-20,118.59,1.39,115.29,0.07,116.68,"[<PayInt:A2>, <PayPrin:A2>]"
2023-02-20,2.71,0.70,115.88,0.07,116.58,"[<PayInt:A2>, <PayPrin:A2>]"


In [19]:
10 / 365 *0.08* 348.69

0.7642520547945205

In [20]:
1.528504109589041+0.7642520547945205

2.2927561643835617